In [1]:
import time
import sys

sys.argv = ['--video', './input_video.mp4']

from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images
from yolov3_tf2.utils import draw_outputs
from IPython.display import clear_output, Image, display, HTML

flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('video', './input_video.mp4',
                    'path to video file or number for webcam)')
flags.DEFINE_string('output', None, 'path to output video')
flags.DEFINE_string('output_format', 'XVID', 'codec used in VideoWriter when saving video to file')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

def main(argvs):
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=64,  inter_op_parallelism_threads=1)
    session = tf.compat.v1.Session(config=config) 
    tf.compat.v1.keras.backend.set_session(session)

    #yolo = YoloV3Tiny(classes=FLAGS.num_classes)
    yolo = YoloV3(classes=FLAGS.num_classes)
    yolo.load_weights(FLAGS.weights)
    print('weights loaded')

    class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
    print('classes loaded')

    times = []

    vid = cv2.VideoCapture('input_video.mp4')

    display_handle = display(None, display_id=True)
    frames = 0
    try:
        while True:
            _, img = vid.read()

            if img is None:
              break

            img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_in = tf.expand_dims(img_in, 0)
            img_in = transform_images(img_in, 416)

            t1 = time.time()
            boxes, scores, classes, nums = yolo.predict(img_in)
            t2 = time.time()
            times.append(t2-t1)
            times = times[-20:]

            img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
            latency = sum(times)/len(times) * 1000
            img = cv2.putText(img, "Time: {:.2f}ms".format(latency), (0, 30),
                   cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
            scale = 0.7
            width = int(img.shape[1] * scale)
            height = int(img.shape[0] * scale)
            img = cv2.resize(img, (width, height))
            _, img = cv2.imencode('.jpeg', img)
            display_handle.update(Image(data=img.tobytes()))
    except KeyboardInterrupt:
        pass
    finally:
        vid.release()
        display_handle.update(None)
    
app.run(main)



weights loaded
classes loaded


None

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


SystemExit: 

/root/miniforge3/envs/tensorflow/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
